 ## 第14章 策略梯度方法
&emsp;&emsp;在之前的章节中，几乎的所有方法都采用动作值方法（action-value methods），即通过学习到的动作值函数来选择动作。本章主要介绍参数化策略方法（parameterized policy），不再利用值函数来选择动作，而是利用策略函数来选择动作，同时用值函数辅助策略函数参数的更新。<br>
&emsp;&emsp;策略梯度法（policy gradient，PG）是参数化策略函数的常用算法，根据策略类型的不同，PG还可以分为随机策略梯度（stochastic policy gradient，SPG）方法和确定性策略梯度（deterministic policy gradient，DPG）方法。
### 12.1 随机策略梯度法
#### 12.1.1 梯度上升算法
&emsp;&emsp;参数化的策略不再是一个概率集合，而是一个可微的函数。定义策略函数 表示t时刻在状态 和参数 下选择动作a的概率：  <br>
<center>$\pi(a \mid s, \boldsymbol{\theta})=P\left\{A_{t}=a \mid S_{t}=s, \boldsymbol{\theta}_{t}=\boldsymbol{\theta}\right\}, \quad \boldsymbol{\theta} \in \mathbb{R}^{d}\tag{12.1}$
</center><br>
其中$\theta$的表示策略参数（策略权重，策略参数向量，policy's parameter vector）：策略函数可以
简记为  $\pi_{\theta}$,  策略函数  $\pi(a \mid s, \boldsymbol{\theta})$  简记为  $\pi_{\theta}(a \mid s)$  。 由于参数化，策略函数  $\pi(a \mid s, \theta)$  可以视为概率密度函数，智能体按照该概率分布进行动作
选择，而为了保证探索性，通常都采用随机策略函数。为了衡量策略函数  $\pi(a \mid s, \boldsymbol{\theta})$  的优劣性，
需要设计一个关于策略参数  $\boldsymbol{\theta}$  的目标函数（performance measure）$J(\boldsymbol{\theta})$（用不同符号来与值函数
逼近法的目标函数 L(w)进行区分）, 最直接的想法就是将目标函数 $J(\boldsymbol{\theta})$定义为折扣回报的期望:

<center>$J(\boldsymbol{\theta})=\mathbb{E}_{\pi_{\theta}}[G]=\mathbb{E}_{\pi_{6}}\left[R_{1}+\gamma R_{2}+\gamma^{2} R_{3}+\cdots+\gamma^{n-1} R_{n}\right]\tag{12.2}$</center><br>
由于定义的区别，$J(\boldsymbol{\theta})$ 与值函数逼近法的损失函数L(w)意义不同，它的策略梯度$\nabla J(\boldsymbol{\theta})$ 是存在无偏估计形式的。<br>
&emsp;&emsp;此时，算法的目标是使得回报最大化，所以对参数$\theta$采用梯度上升方法，该方法也被称为
随机梯度上升（stochastic gradient-ascent，SGA）算法。基于 SGA 的参数$\theta$ 的更新方程如下所示:

<center>$\boldsymbol{\theta} \leftarrow \boldsymbol{\theta}+\alpha \nabla \hat{J}(\boldsymbol{\theta}), \quad \nabla \hat{J}(\boldsymbol{\theta}) \in \mathbb{R}^{d^{*}}\tag{12.3}$</center><br>
&emsp;&emsp;其中$\nabla \hat{J}(\boldsymbol{\theta})$为策略梯度（策略梯度函数，policy gradient、policy gradient function）估计值，也就是近似策略梯度；策略参数$\theta$与值函数逼近中的权重参数W作用相同，使用不同的符号仅有助于区分所用方法；维度$d^{\prime}$ 也仅用于区分值函数逼近参数w 的维度d。<br>
&emsp;&emsp;所有满足式(12,3)的方法都称为SPG，但在不区分DPG的情况下，通常会直接称SPG为PG。PG是一大类基于梯度的策略学习方法，后文将要介绍的REINFORCE算法和行动者-评论家（演员-评论家、行动器-评判器，actor–critic，AC）算法都属于该方法。<br>

####  12.1.2 策略梯度法与值函数逼近法的比较

&emsp;&emsp;从值函数逼近法出发，本节讨论PG法的优点和缺点：<br>
**（1）PG法的优点**
&emsp;&emsp;（1）平滑收敛：在学习过程中，PG法每次更新策略函数，权重参数都会朝着最优值变化，且只发生微小变化，有很强的收敛性；值函数逼近法基于贪婪策略对策略进行改进，有些价值函数在后期会一直围绕最优价值函数持续小的震荡而不收敛，即策略退化（policy degradation）现象。<br>
&emsp;&emsp;（2）处理连续动作空间任务：PG法可以直接得到一个策略；值函数逼近法需要对比状态s所有动作的价值，才能得到最优动作值函数 $q_{*}(s, a)$，在处理大动作空间或连续状态动作空间任务时，难以实现。<br>
&emsp;&emsp;（3）学习随机策略：在实际问题中，策略往往都是随机的，PG法能够输出随机策略；值函数逼近法基于贪婪方法，每次输出的都是确定性策略。有时随机策略才是最优策略：比如“剪刀石头布”游戏，如果按照某一种策略来出拳，很容易让对手抓住规律，输掉游戏；最好的策略就是随机出拳，让对手猜不到。<br>
**（2）PG法的缺点**
&emsp;&emsp;（1）PG法通常只能收敛到局部最优解。<br>
&emsp;&emsp; （2）PG法的易收敛性和学习过程平滑优势，都会使智能体尝试过多无效的探索，从而造成学习效率低，整体策略方差偏大，以及存在累积误差带来的过高估计问题<br>
&emsp;&emsp;**例 12.1** 以格子世界为例，考虑算法只能输出确定性策略的缺陷。如下图a所示格子世界，智能体需要在不进入陷阱区域X的情况下到达目标G，智能体的初始状态会随机出现在第一行的5个格子中。<br>
<center><img src='./第十二章图片/图12.1.png'></center>
<center>图12.1&ensp; 关于确定性策略的缺陷问题</center>
&emsp;&emsp;如果采用坐标信息来表示格子，可以直接使用值函数方法来求解；但如果采用“格子某个方向是否有墙”这样的特征信息来描述格子，就会出现如图b所示的情况，两个灰色格子的状态特征描述是一样的，即发生了重名现象（aliased）：<br>
<center>$ x_{\theta}$(  南北有墙,往荣走  )=1  or 0</center><br>
<center>$ \pi_{0}$(  南北有墙,往西走  )=0  or 1</center>
&emsp;&emsp;实际上，当智能体处于这两个灰色格子时应该采取的动作是不一样的，所以一旦出现重名现象，随机策略就会比确定性策略效果更好：<br>
<center>$\pi_{\theta}$(南北有墙 , 往东走 )=0.5</center> <br>
<center>$\pi_{\theta}$( 南北有墙,往西走)=0.5</center><br>

&emsp;&emsp;**例 12.2**以格子世界为例，比较值函数逼近法和策略梯度法学习到的策略。如下图所示，S为起点，G为终点，阴影表示障碍物，动作A={上，下，左，右}，智能体若离开边界或撞到障碍物都会返回上一个位置，到达目标G时奖励为1，其他转移情况均为0，其他参数 $\gamma$=0.95，$\alpha$=0.1 ，$\mathcal{E}$=0.1 。<br>
<center><img src='./第十二章图片/图12.2.png'></center>
<center>图12.2&ensp; 值函数逼近法和策略梯度法学习到的策略</center>
&emsp;&emsp;多次迭代后，值函数逼近法和策略梯度法学习到的策略轨迹分别如图b和图c所示。图b显示，智能体只能获得一条路径；而图c显示，智能体能够得到两条路径。<br>

### 12.2 策略优化方法

&emsp;&emsp;策略优化方法与值函数逼近法一样，需要分为情节式任务和持续性任务来考虑。<br>

#### 12.2.1 情节式策略目标函数

&emsp;&emsp;由上已知，PG法的目标为最大化目标函数 $J(\boldsymbol{\theta})$。针对情节式任务，又可以基于离散状态-动作空间任务和连续状态-动作空间任务，来分别定义不同的目标函数：<br>
**（1）初始价值（start value）**<br>
&emsp;&emsp;初始价值适用于离散状态-动作空间任务，假设每个情节都从初始状态$s_{0}$ 开始，其目标函数由初始状态价值的期望构成：<br>
<center>$J_{\pi}(\boldsymbol{\theta})=v_{\pi_{\theta}}\left(s_{0}\right)\tag{12.4}$</center><br>
其中，$\mid v_{\pi_{\theta}}$表示策略函数$\pi_{\theta}$的真实状态值函数，有的资料会将$v_{\pi_{\theta}}$记为$v^{\pi_{\theta}}$。<br>

**（2）平均价值（average value）**<br>
&emsp;&emsp;平均价值适用于连续状态-动作空间任务，在该任务中，智能体不存在初始状态 $s_{0}$，所以平均价值计算的是t时刻下所有可能状态的价值，与t时刻的状态分布概率 $\rho^{\pi_{\theta}}(s)$加权和：<br>
<center>$J_{\text {agv }}(\boldsymbol{\theta})=\sum_{s \in \mathcal{S}} \rho^{\pi_{0}}(s) v_{\pi_{0}}(s)\tag{12.5}$</center>

**（3）时间步平均奖励（average reward per time-step value）**<br>
&emsp;&emsp;时间步平均奖励适用于连续状态-动作空间任务，这是一种使用1-步TD(0)算法的方法，它计算t时刻的奖励期望：<br>
<center>$\begin{aligned}
J_{\text {ayg }}(\boldsymbol{\theta}) &=\mathbb{E}_{\pi_{\theta}}[r] \\
&=\sum_{s \in S} \rho^{\pi_{\theta}}(s) \sum_{a \in \mathcal{A}} \pi(a \mid s, \boldsymbol{\theta}) r
\end{aligned}\tag{12.6}$</center>

#### 12.2.2 持续性策略目标函数
&emsp;&emsp;在无法使用情节式边界的持续性任务中，根据每个时刻的平均回报 来定义目标函数：
<center>$\begin{aligned}
J_{c}(\boldsymbol{\theta}) &=r(\pi) \\
&=\lim _{t \rightarrow \infty} \mathbb{E}\left[R_{t} \mid S_{0}, A_{0 t-1} \sim \pi\right] \\
&=\sum_{s} \rho^{\pi_{\theta}}(s) \sum_{a} \pi(a \mid s) \sum_{z^{\prime}, r} p\left(s^{\prime}, r \mid s, a\right) r
\end{aligned}\tag{12.7}$</center><br>
&emsp;&emsp;其中$\rho^{\pi_{\theta}}(s)=\lim _{t \rightarrow \infty} P\left(S_{t}=s \mid A_{0 t} \sim \pi\right)$满足遍历性假设。<br>

#### 12.3 策略梯度定理
&emsp;&emsp;原则上，我们可以直接对式(12,2)的目标函数 $J(\boldsymbol{\theta})$求梯度，再利用SGA优化参数$\boldsymbol{\theta}$,但是基于回报期望的目标函数和策略函数的联系并不直观，这样的目标函数梯度难以直接用于参数优化。从另一个角度考虑，对于连续状态-动作空间任务来说，除了动作的选择，状态分布$\rho^{\pi_{\theta}}(s)$也受到策略参数$\boldsymbol{\theta}$ 的影响，虽然可以通过策略参数$\boldsymbol{\theta}$ 计算出动作选择概率 $\pi(a \mid s, \boldsymbol{\theta})$和相应奖励$r(a \mid s)$,但因为状态分布$\rho^{\pi_{6}}(s)$与环境有关，所以无法确定策略梯度$\nabla J(\boldsymbol{\theta})$与状态分布$\rho^{\pi_{\theta}}(s)$之间的关系。由此，我们需要对目标函数进行调整。<br>
**（1）全部动作算法下的策略梯度定理**<br>
&emsp;&emsp;一个直观的想法是，如果执行某一个动作能够得到更多奖励（或回报，或值函数），那么就应该增加它出现的概率，反之减小。基于这一想法，我们考虑最简单的1-步TD(0)情况，构建一个与策略参数$\boldsymbol{\theta}$无关的评价指标函数f(s, a)，用于测量在状态s下采取动作a可以获得的奖励（或回报，或值函数），以此得到基于评价指标期望的目标函数如下所示：<br>
<center>$\begin{aligned}
&J(\boldsymbol{\theta})=\mathbb{E}_{\pi_{\theta}}(f(s, a))\\
&=\sum_{s \in S} \rho^{\pi_{\theta}}(s) \sum_{a \in \mathcal{A}} \pi(a \mid s, \boldsymbol{\theta}) f(s, a) \quad \text { 离散空间任务 }\\
&=\int_{S} \rho^{\pi_{0}}(s) \int_{\mathcal{A}} \pi(a \mid s, \boldsymbol{\theta}) f(s, a) d a d s \quad \text { 连续空间任务 }
\end{aligned}$</center><br>
其中，状态分布$\rho^{\pi_{\theta}}(s)$是策略函数$\pi_{\theta}$下的同策略分布。<br>
&emsp;&emsp;以离散空间任务为例，为了构建一个仅对策略参数$\boldsymbol{\theta}$求导，而不涉及对状态分布$\rho^{\pi_{\theta}}(s)$求导的目标函数导数形式，将状态作为分布函数：<br>
<center>$\begin{aligned}
\nabla J(\boldsymbol{\theta}) &=\nabla \sum_{s \in S} \rho^{\pi_{6}}(s) \sum_{a \in \mathcal{A}} \pi(a \mid s, \boldsymbol{\theta}) f(s, a) \\
&=\sum_{s \in \mathcal{S}} \rho^{\pi_{\theta}}(s) \sum_{a \in \mathcal{A}} \nabla \pi(a \mid s, \boldsymbol{\theta}) f(s, a)\\
&=\mathbb{E}_{S_{i} \sim \rho^{r_{0}}}\left[\sum_{a \in \mathcal{A}} \nabla \pi\left(a \mid S_{t}, \boldsymbol{\theta}\right) f\left(S_{t}, a\right)\right] \quad \text { 将 } \sum_{s \in S} \rho^{\pi_{0}}(s) \text { 作为期望概率 }
\end{aligned} \\\tag{12.8}$</center><br>

由于该算法涉及所有可能的动作，所以也被称为全部动作算法（all-actions method）。有的资料会将$\mathbb{E}_{S_{t} \sim \rho^{r\theta}} \dot{\mathbb{H}}$记为$\mathbb{E}_{\pi_{\theta}}$。<br>
&emsp;&emsp;正如之前所说的，评价指标f(s, a)可以用奖励r,回报G或值函数等形式来表示，常用的评价指标为动作值函数$q_{\pi_{\theta}}(s, a)$，其策略梯度如下所示：<br>

<center>$\nabla J(\boldsymbol{\theta})=\mathbb{E}_{S_{i} \sim p^{\prime 6}}\left[\sum_{a \in \mathcal{A}} \nabla \pi\left(a \mid S_{t}, \boldsymbol{\theta}\right) q_{\pi_{\theta}}\left(S_{t}, a\right)\right]\tag{12.9}$</center><br>
该式也被称为策略梯度定理（policy gradient theorem），该定理同时适用于离散和连续状态-动作空间任务，也就是说，前两节介绍的四种目标函数$ J_{\mathrm{sv}}$, $J_{\text {argv }}$, $J_{\text {arg }}$, $J_{c} $都可以采用这一策略梯度。<br>
&emsp;&emsp;由此得到策略参数更新方程如下所示：<br>
<center>$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_{t}+\alpha \sum_{a \in \mathcal{A}} \nabla \pi\left(a \mid S_{t}, \boldsymbol{\theta}_{t}\right) q_{\pi_{\theta}}\left(S_{t}, a\right)\tag{12.10}$</center><br>
**（2）单步算法下的策略梯度定理**
&emsp;&emsp;在实际情况下，由于需要进行采样，策略梯度定理通常仅考虑采样得到动作$A_{t}$。基于式(12,8)，将动作作为分布函数，并保证不改变梯度大小：<br>

<center>$  
\begin{aligned}
\nabla J(\boldsymbol{\theta})&=\mathbb{E}_{S_{t} \sim \rho^{\pi_{\theta}}}\left[\sum_{a \in \mathcal{A}} \pi\left(a \mid S_{t}, \boldsymbol{\theta}\right) \frac{\nabla \pi\left(a \mid S_{t}, \boldsymbol{\theta}\right)}{\pi\left(a \mid S_{t}, \boldsymbol{\theta}\right)} f\left(S_{t}, a\right)\right] \quad  先乘后除保证等价性\\
&=\mathbb{E}_{S_{t} \sim \rho^{\pi_{\theta}}, A_{t} \sim \pi_{\theta}}\left[\frac{\nabla \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right)}{\pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right)} f\left(S_{t}, A_{t}\right)\right] \\
 &=\mathbb{E}_{S_{t} \sim \rho^{\pi_{\theta}}, A_{t} \sim \pi_{\theta}}\left[\nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) f\left(S_{t}, A_{t}\right)\right] 
用齐样数据  A_{t} \sim \pi  替换  a  
\end{aligned} \\\tag{12.11}$</center><br>

其中，策略梯度$\nabla \pi(a \mid s, \boldsymbol{\theta})$或其对数$\nabla \log \pi(a \mid s, \boldsymbol{\theta})$被称为迹向量（eligibility vector），表示参数空间中在访问状态$S_{t}$时最能增加重复动作的概率的方向。直观理解上式，就是评价指标期望越大的动作，就让它出现的概率（即迹向量）越大，反之越小。<br>


&emsp;&emsp;同理，将动作值函数$q_{\pi_{\theta}}(s, a)$作为评价指标，策略梯度法也可以表示成如下形式：<br>

<center>$\nabla J(\boldsymbol{\theta})=\mathbb{E}_{S_{t} \sim \rho^{* \theta}, A_{t} \sim \pi_{\theta}}\left[\nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) q_{\pi_{\theta}}\left(S_{t}, A_{t}\right)\right]\tag{12.12}$</center><br>

上式表示，动作值函数期望越高的动作，其出现概率也应该越高。有的资料将$S_{t}, A_{t}$记为s,a。
&emsp;&emsp;现在，我们的目标转为求解迹向量$\nabla \log \pi(a \mid s, \boldsymbol{\theta})$和动作值函数$q_{\pi_{\theta}}(s, a)$。<br>

### 12.3 策略表达形式

&emsp;&emsp;为了求迹向量$\nabla \log \pi(a \mid s, \boldsymbol{\theta})$，首先需要构建策略函数的参数表达形式。在PG法中，只要策略函数$\pi(a \mid s, \boldsymbol{\theta})$可导，策略就可以用任意的方式参数化。与值函数逼近法一样，策略函数 也需要分成小型离散动作空间（softmax函数）和大型或连续动作空间（高斯策略函数）两种情况讨论。<br>

#### 12.3.1 离散动作空间策略参数化

&emsp;&emsp;对于小型离散动作空间问题，对每一组状态-动作对都估计一个动作偏好值（偏好值，参数化数值偏好，parameterized numerical preferences）$h(s, a, \boldsymbol{\theta}) \in \mathbb{R}$ ，也就是特征函数。动作偏好值$h(s, a, \boldsymbol{\theta})$可以用任意的方式参数化，通常将它视为多个特征的线性加权之和：<br>

<center>$h(s, a, \boldsymbol{\theta})=\boldsymbol{\theta}^{\mathrm{T}} \mathbf{x}(s, a)\tag{12.13}$</center><br>

其中，$\mathbf{x}(s, a) \in \mathbb{R}^{d^{\prime}}$ 表示特征向量。在某个状态下动作表现越好，其偏好值$h(s, a, \boldsymbol{\theta})$就越高；若最优策略是确定性策略，则相对于次优动作，其偏好值将趋于无穷大。<br>

由此可知，策略函数$\pi(a \mid s, \boldsymbol{\theta})$正比于动作偏好值$h(s, a, \boldsymbol{\theta})$:<br>

<center>$\pi(a \mid s, \boldsymbol{\theta}) \propto e^{h(s, a, \boldsymbol{\theta})}$</center><br>

采用指数柔性最大化分布（softmax函数，exponential soft-max distribution）构建基于动作偏好值的策略函数，即softmax策略函数，输出状态s下所有可执行动作的概率分布：<br>

<center>$\pi(a \mid s, \boldsymbol{\theta})=\frac{e^{h(s, a, \boldsymbol{\theta})}}{\sum_{a^{\prime}} e^{h\left(s, a^{\prime}, \boldsymbol{\theta}\right)}}\tag{12.14}$</center><br>

其中，分母用于归一化操作，使得每个状态选择动作的概率之和为1。<br>

&emsp;&emsp;softmax策略的迹向量如下所示：<br>

<center>$\begin{aligned}
\nabla \log \pi(a \mid s, \boldsymbol{\theta}) &=\mathbf{x}(s, a)-\mathbb{E}_{\pi_{\theta}}[\mathbf{x}(s, \cdot)] \\
&=\mathbf{x}(s, a)-\sum_{a \in \mathcal{A}} \pi(s, a, \boldsymbol{\theta}) \mathbf{x}(s, a)
\end{aligned}\tag{12.15}$</center><br>

其中，$\mathbf{x}(s, a)$表示在状态s下，采取动作a的得分；$\mathbb{E}_{\pi_{\theta}}[\mathbf{x}(s, \cdot)]$表示在状态s的期望分值。

#### 12.3.2 连续动作空间策略参数化

对于大型离散动作空间或连续动作空间问题，PG法不直接计算每一个动作被选择的概率，而是学习动作概率分布，根据高斯分布（正态分布，Gaussian、normal distribution）来选择动作。高斯分布的概率密度函数为：<br>

<center>$p(x)=\frac{1}{\sqrt{2 \pi} \sigma} e^{-\frac{(x-\mu)^{2}}{2 \sigma^{2}}}$</center><br>

其中，$\mu$和$\sigma$分别为高斯分布的均值和标准差；$p(\mathrm{X} \leq x)$表示小于x点的图像所围成的面积，$p(x)$图像下的总面积恒为1。<br>

&emsp;&emsp;将策略函数定义为实数型动作的正态概率密度：<br>

<center>$\pi(a \mid s, \boldsymbol{\theta})=\frac{1}{\sqrt{2 \pi} \sigma} e^{-\frac{(a-\mu(s, \theta))^{2}}{2 \sigma^{2}}}\tag{12.16}$</center><br>

其中，$\mu: \mathcal{S} \times \mathbb{R}^{d^{\prime}} \rightarrow \mathbb{R}$通常用一个线性函数来逼近：$\mu(s, \boldsymbol{\theta})=\boldsymbol{\theta}^{\mathrm{T}} \mathbf{x}(s) ; \sigma: \mathcal{S} \times \mathbb{R}^{d^{\prime}} \rightarrow \mathbb{R}$则设置为一个固定正数。满足该式的策略函数称为高斯策略函数。


&emsp;&emsp;高斯策略的迹向量如下所示：<br>
<center>$\nabla \log \pi(a \mid s, \boldsymbol{\theta})=\frac{(a-\mu(s, \boldsymbol{\theta})) \mathbf{x}(s)}{\sigma^{2}}\tag{12.17}$</center><br>

### 12.4 蒙特卡洛策略梯度法

动作值函数$q_{\pi_{\theta}}(s, a)$可以通过DP、MC、TD等基础强化学习算法进行学习。蒙特卡洛策略梯度法（REINFORCE）是一种针对情节式问题的，基于MC算法的PG法，本节将介绍REINFORCE算法和带基线的REINFORCE算法。<br>

#### 12.4.1 REINFORCE

REINFORCE算法采用MC算法来计算动作值函数，只考虑智能体在状态$S_{t}$下实际采取的动作$A_{t}$:<br>
<center>$\begin{aligned}
\nabla J(\boldsymbol{\theta}) &=\mathbb{E}_{S_{t} \sim \rho^{\pi_{\theta}}, A_{t} \sim \pi_{\theta}}\left[\nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) q_{\pi_{\theta}}\left(S_{t}, A_{t}\right)\right] \\
&=\mathbb{E}_{S_{t} \sim \rho^{\pi \theta}, A_{t} \sim \pi_{\theta}}\left[\nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) G_{t}\right] & \text { 根据 } q_{\pi_{\theta}}\left(S_{t}, A_{t}\right)=\mathbb{E}_{\pi_{\theta}}\left[G_{t} \mid S_{t}, A_{t}\right]
\end{aligned}\tag{12.18}$</center><br>

由于采用的是MC算法，所以这是一种对策略梯度$\nabla J(\boldsymbol{\theta})$的无偏估计。
&emsp;&emsp;得到REINFORCE算法的策略参数 更新方程如下所示：<br>

<center>$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_{t}+\alpha \nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}_{t}\right) G_{t}\tag{12.19}$</center><br>


上式可以从理论上保证策略参数$\theta$的收敛性，最大化$J(\boldsymbol{\theta})$：（1）梯度增量$\Delta \boldsymbol{\theta}$正比于回报$G_{t}$，使得策略参数$\theta$向着能够产生最大回报的动作的方向更新；（2）梯度增量$\Delta \boldsymbol{\theta}$ 反比于迹向量，能够减少频繁被选择的动作。<br>

&emsp;&emsp;用于估算最优策略的，REINFORCE算法的伪代码如下：

伪代码 12.1用于估算最优策略的，REINFORCE算法

输入：可微策略函数 

初始化：<br>
1. $\alpha>0 \mid$<br>
2. 初始化策略参数$\boldsymbol{\theta} \in \mathbb{R}^{d^{\prime}}$

3. Loop（每个情节）：<br>
4. &emsp;&emsp;根据策略 生成一个情节 $S_{0}, A_{0}, R_{1}, \ldots, S_{T-1}, A_{T-1} R_{T}$<br>
5. &emsp;&emsp;$G \leftarrow 0 $<br>
6. &emsp;&emsp;Loop 情节中的所有时刻，$t=T-1, T-2, \ldots, 0:$<br>
7. &emsp;&emsp; &emsp;&emsp;$G \leftarrow \gamma G+R_{t+1}$<br>
8. &emsp;&emsp; &emsp;&emsp;$\boldsymbol{\theta} \leftarrow \boldsymbol{\theta}+\alpha \gamma^{t} \nabla \ln \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) G$

对伪代码 12.1中的部分行代码进行说明：<br>
1. 步长<br>
2. $\theta$ 通常初始化为0向量<br>
4. 采用MC算法，所以获取的是完整的情节<br>
7. 在实际应用中，会使用带折扣系数的回报<br>
8. 对策略参数$\theta$ 进行更新。注意这里多了一个折扣参数$\gamma^{t}$，配合上折扣回报，再次缩小参数更新幅度。

#### 12.4.2 带基线的REINFORCE

&emsp;&emsp;REINFORCE的优势在于，只需要很小的更新步长就能收敛到局部最优，并保证了每次的更新都是有利的。但是假设每个动作的奖励均为正（即所有的策略梯度值$\nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta})$均大于或等于零时），则每个动作出现的概率将不断地提高，这一现象会严重降低学习速率，并增大梯度方差。<br>
考虑一个随机变量X，其方差为$\mathrm{DX}=\mathrm{EX}^{2}-(\mathrm{EX})^{2}$，如果能够使$\mathrm{EX}^{2}$ 减小，那么方差 也会减小，最直接的做法就是让X减去一个值。根据这一思想，我们构建一个仅与状态有关的基线函数（baseline）$b(s)$ ，保证能够在不改变策略梯度$\nabla J(\boldsymbol{\theta})$的同时，降低其方差。当$b(s)$ 具备上述特点时，下面的推导成立：<br>
<center>$\begin{aligned}
&\mathbb{E}_{\pi_{\theta}}\left[\nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) b\left(S_{t}\right)\right]=\sum_{s \in \mathcal{S}} \rho^{\pi_{\theta}}(s) \sum_{a \in \mathcal{A}} \nabla \pi(a \mid s, \boldsymbol{\theta}) b(s) \quad \text { 展开期望公式 }\\
&=\sum_{s \in \mathcal{S}} \rho^{\pi_{\theta}}(s) b(s) \nabla \sum_{a \in \mathcal{A}} \pi(a \mid s, \boldsymbol{\theta}) \quad b(s) \text { 与动作无关 } ; \text { 提出梯度 }\\
&=0 \quad \sum_{a \in \mathcal{A}} \pi(a \mid s, \boldsymbol{\theta})=1 \Rightarrow \nabla \sum_{a \in \mathcal{A}} \pi(a \mid s, \boldsymbol{\theta})=0
\end{aligned}$</center><br>

由此可见，为评价指标增加基线$b(s)$并不会改变策略梯度$\nabla J(\boldsymbol{\theta})$，所以带基线的强化学习方法是无偏差的。原则上，和动作无关的任意函数或变量都可作为$b(s)$。<br>

&emsp;&emsp;带基线的REINFORCE算法是REINFORCE算法的改进版，策略梯度计算公式如下所示：<br>

<center>$\nabla J(\boldsymbol{\theta})=\mathbb{E}_{S_{t} \sim \rho^{\pi_{\theta}}, A_{t} \sim \pi_{\theta}}\left[\nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right)\left(G_{t}-b\left(S_{t}\right)\right)\right]\tag{12.20}$</center><br>

带基线的策略参数$\theta$的更新方程如下所示：<br>
<center>$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_{t}+\alpha \nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}_{t}\right)\left(G_{t}-b\left(S_{t}\right)\right)\tag{12.21}$</center><br>

当$b\left(S_{t}\right)$为0时，该式就是式(12,19)。<br>
&emsp;&emsp;原则上，和动作无关的函数都可以作为基线$b(s)$ 。但是为了有效地利用基线，对所有动作值都比较大的状态，需要设置一个较大的基线来区分最优动作和次优动作；对所有动作值都比较小的状态，则需要设置一个比较小的基线。由此，用近似状态值函数$\hat{v}(s, \mathbf{w})$代表基线$b(s)$ ，当回报超过基线值时，该动作的概率将提高，反之降低：<br>

<center>$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_{t}+\alpha \nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}_{t}\right)\left(G_{t}-\hat{v}\left(S_{t}, \mathbf{w}\right)\right)\tag{12.22}$</center><br>

&emsp;&emsp;用于估算最优策略的，带基线的REINFORCE算法的伪代码如下：<br>
伪代码 12.2用于估算最优策略的，带基线的REINFORCE算法<br>
输入：可微策略函数$\pi(a \mid s, \boldsymbol{\theta})$<br>
&emsp;&emsp;&emsp;可微近似状态值函数 ，作为基线<br>
初始化：<br>
1. 初始化步长$\alpha^{\theta}>0, \quad \alpha^{\mathrm{w}}>0$<br>
2. 初始化策略参数$\boldsymbol{\theta} \in \mathbb{R}^{d^{\prime}} \text { 和状态值函数权重 } \mathbf{w} \in \mathbb{R}^{d}$<br>
3. Loop（每个情节）：<br>
4.   &emsp;&emsp; 根据策略$\pi(\cdot \mid \cdot, \boldsymbol{\theta}) \text { 生成一个情节 } S_{0}, A_{0}, R_{1}, \ldots, S_{T-1}, A_{T-1,} R_{T}$<br>
5. &emsp;&emsp;$G \leftarrow 0$<br>
6.   &emsp;&emsp;  Loop 情节中的所有时刻，$t=T-1, T-2, \ldots, 0:$<br>
7.  &emsp;&emsp;  &emsp;&emsp; $G \leftarrow \gamma G+R_{t+1}$<br>

8.  &emsp;&emsp;  &emsp;&emsp;$\delta \leftarrow G-\hat{v}\left(S_{t}, \mathbf{w}\right)$<br>


9.  &emsp;&emsp;  &emsp;&emsp;$\mathbf{w} \leftarrow \mathbf{w}+\alpha^{\mathrm{w}} \delta \nabla_{\mathrm{w}} \hat{v}\left(S_{t}, \mathbf{w}\right)$<br>
10.  &emsp;&emsp;  &emsp;&emsp;$\boldsymbol{\theta} \leftarrow \boldsymbol{\theta}+\alpha^{\theta} \gamma^{t} \nabla_{\theta} \ln \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) \delta$<br>

 &emsp;&emsp;对伪代码 12.1中的部分行代码进行说明：<br>
&emsp;&emsp; 1. 由于带基线的REINFORCE算法同时使用到了PG法和状态值函数逼近法，所以需要分别设定策略梯度的步长$\alpha^{\theta}$和近似状态值函数的步长$\alpha^{\mathrm{w}}$<br>
&emsp;&emsp; 8. 近似状态值函数$\hat{\mathcal{V}}$ 作为基线，更新MC算法的目标值<br>
&emsp;&emsp;9.采用基于SGD的函数逼近算法，更新近似值函数参数 <br>
&emsp;&emsp;10. 采用PG法，更新策略参数$\theta$<br>
&emsp;&emsp;从这里开始，经常会在一个算法中涉及到对策略参数$\theta$和值函数参数w的更新，所以在求导时，使用$\nabla_{\theta} \text { 和 } \nabla_{\mathrm{w}}$来区分。<br>

### 12.5 行动者-评论家

&emsp;&emsp;REINFORCE都采用回合更新方法，虽然是无偏的，但是方差高。一种直观的改进方式就是采用自举的更新方法，在每一步或几步之后即时地做出策略改进，虽然引入了偏差，但可以有效减小方差。<br>
&emsp;&emsp;行动者-评论家（演员-评论家、行动器-评判器，actor–critic，AC）算法正是这样一种利用了自举的方法，它将PG法（策略网络）和值函数逼近法（值函数网络）相结合，同时学习策略和值函数，实现实时、在线的学习：<br>
    &emsp;&emsp;（1）一方面，行动者（actor）依赖于评论家（critic）的值函数，利用PG法更新策略参数$\theta$，学习（改进）策略$\pi_{\theta}$；<br>
     &emsp;&emsp;（2）另一方面，评论家依赖于行动者策略$\pi_{\theta}$得到的经验样本，利用值函数逼近法更新值函数参数w，学习（改进）近似值函数$\hat{v}(s, \mathbf{w}) \text { 或 } \hat{q}(s, a, \mathbf{w})$。<br>
&emsp;&emsp;AC基本框架图如下所示：<br>
<center><img src='./第十二章图片/图12.3.png'></center>
<center>图12.3&ensp; AC基本框架图</center>
直观来理解，AC过程如下所示：<br>
（1）智能体根据游戏的当前状态选择一个动作（基于当前策略或初始化策略）；<br>
（2）评论家根据当前状态-动作对，对当前策略的表现打分；<br>
（3）行动者依据评论家的打分，改进策略（调整策略参数）；<br>
（4）评论家根据环境返回的奖励，改进策略打分方式（调整值函数参数）；<br>
（5）利用更新后的策略在下一状态处选择动作，重复以上过程<br>
&emsp;&emsp;刚开始时，行动者随机选择动作，评论家随机打分。但由于环境返回的奖励，评论家的评分会越来越准确，行动者会选择到更好的动作。<br>
&emsp;&emsp;需要注意的是，带基线的REINFORCE算法虽然也同时运用了值函数和策略函数，但它并不属于AC方法，因为它的值函数仅仅作为基线，而不是评论家。<br>

#### 12.5.1 行动者-评论家的评价指标

&emsp;&emsp;由于AC框架使用的是基于自举的PG法，所以需要确定它的评价指标形式。<br>
&emsp;&emsp;**（1）优势函数**<br>
&emsp;&emsp;根据基线的思想，将状态s的价值$v_{\pi_{\theta}}(s)$作为基线$b(s)$，定义优势函数（动作优势函数，Advantage function）$A_{\pi_{\theta}}(s, a)$如下所示：<br>
<center>$A_{\pi_{\theta}}(s, a)=q_{\pi_{\theta}}(s, a)-v_{\pi_{\theta}}(s)\tag{12.23}$</center><br>

其中，优势函数$A(s, a)$表示在状态s处选择动作a时所得的到回报，相比于采取所有可能动作所带来的平均回报的差异；也就是说，优势函数指的是当前状态-动作对(s,a)的价值相比于当前状态s的价值的优势。此外，也有另一种说法，优势函数$A(s, a)$表示在状态s处选择动作a时，相比于选择最优动作$a_{*}$能带来的收益，即选择动作a比选择最优动作$a_{*}$能好多少，即令$A\left(s, a_{*}\right)=0$。<br>
&emsp;&emsp;利用优势函数$A(s, a)$，策略梯度$\nabla_{\theta} J(\boldsymbol{\theta})$可以表示为如下形式：<br>

<center>$\begin{aligned}
\nabla_{\mathbf{\theta}} J(\boldsymbol{\theta}) &=\mathbb{E}_{S_{t} \sim \rho^{\pi_{\theta}}, A_{t} \sim \pi_{\theta}}\left[\nabla_{\boldsymbol{\theta}} \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right)\left(q_{\pi_{\theta}}\left(S_{t}, A_{t}\right)-v_{\pi_{\theta}}\left(S_{t}\right)\right)\right] \\
&=\mathbb{E}_{S_{t} \sim \rho_{\theta}, A_{t} \sim \pi_{\theta}}\left[\nabla_{\boldsymbol{\theta}} \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) A_{\pi_{\theta}}\left(S_{t}, A_{t}\right)\right]
\end{aligned}\tag{3.24}$</center><br>

实际上，$A_{\pi_{\theta}}(s, a)$也是评价指标f(s,a)的一种形式。由于这里的$A_{\pi_{\theta}}$是真实优势函数，所以$\nabla_{\theta} J(\boldsymbol{\theta})$是策略梯度的无偏估计。<br>
&emsp;&emsp;利用优势函数可以有效减小策略梯度$\nabla_{\theta} J(\boldsymbol{\theta})$的方差，但此时需要使用两个近似值函数参数：<br>
<center>$\begin{array}{l}
\hat{v}\left(s, \mathbf{w}^{v}\right) \approx v_{\pi_{\theta}}(s) \\
\hat{q}\left(s, a, \mathbf{w}^{q}\right) \approx q_{\pi_{\theta}}(s) \\
\hat{A}\left(s, a, \mathbf{w}^{q}, \mathbf{w}^{v}\right)=\hat{q}\left(s, a, \mathbf{w}^{q}\right)-\hat{v}\left(s, \mathbf{w}^{v}\right)
\end{array}$</center><br>

其中，$\mathbf{w}^{v}$表示近似状态值函数的参数，$\mathbf{w}^{q}$表示近似动作值函数的参数，$\hat{A}$表示近似优势函数。<br>
&emsp;&emsp;**（2）TD误差**<br>
&emsp;&emsp;由于AC采用了自举方法，为简化算法中出现两个近似值函数参数的情况，我们尝试将优势函数与TD误差建立联系。根据值函数定义，推导优势函数与TD误差$\delta_{\pi_{\theta}}$的关系如下所示：<br>

<center>$\begin{aligned}
A_{\pi_{\theta}}(s, a) &=q_{\pi_{\theta}}(s, a)-v_{\pi_{\theta}}(s) \\
&=\mathbb{E}_{\pi_{\theta}}\left[r+\gamma v_{\pi_{\theta}}\left(s^{\prime}\right) \mid s, a\right]-\mathbb{E}_{\pi_{\theta}}\left[v_{\pi_{\theta}}(s) \mid s\right] \\
&=\mathbb{E}_{\pi_{\theta}}\left[r+\gamma v_{\pi_{\theta}}\left(s^{\prime}\right)-v_{\pi_{\theta}}(s) \mid s, a\right] \\
&=\mathbb{E}_{\pi_{\theta}}\left[\delta_{\pi_{\theta}} \mid s, a\right]
\end{aligned}\tag{12.25}$</center><br>

由此可见，优势函数$A_{\pi_{\theta}}(s, a)$就是TD误差$\delta_{\pi_{\theta}}$的无偏估计（期望）。由此，在式(12,24)中用TD误差$\delta_{\pi_{\theta}}$ 替代优势函数$A_{\pi_{\theta}}(s, a)$：<br>

<center>$\nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta})=\mathbb{E}_{S_{t} \sim \rho^{\pi_{\theta}}, A_{t} \sim \pi_{\theta}}\left[\nabla_{\boldsymbol{\theta}} \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) \delta_{\pi_{\theta}}\right]\tag{3.26}$</center><br>

在实际运用时，采用近似TD误差$\delta_{\mathrm{w}}$:<br>

<center>$\delta_{\mathrm{w}}=A(s, a, \mathbf{w})=r+\gamma \hat{v}\left(s^{\prime}, \mathbf{w}\right)-\hat{v}(s, \mathbf{w})$</center><br>

这样做的好处就是，只需要一组参数就可以描述优势函数。<br>


### 12.6 确定性策略梯度定理

&emsp;&emsp;与确定性策略相比，随机策略自带探索属性，它可以通过探索产生多样的采样数据，并由强化学习算法来改进当前策略。此外，SPG理论相对比较成熟，计算过程更为简单。而采用DPG法时，在初始状态已知的情况下，用确定性策略所产生的轨迹是固定的，也就是说，Agent无法学习。为了在确定性策略下实现对环境的探索，确定性策略需要采用AC框架，并利用异策略学习方法，设置行动策略为随机策略。<br>

&emsp;&emsp;**（1）同策略SPG与DPG**<br>
&emsp;&emsp;回顾SPG计算公式如下所示：<br>

<center>$\nabla J(\boldsymbol{\theta})=\mathbb{E}_{S_{t} \sim \rho^{\pi_{\theta}}, A_{t} \sim \pi_{\theta}}\left[\nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) q_{\pi_{\theta}}\left(S_{t}, A_{t}\right)\right]\tag{12.27}$</center><br>

&emsp;&emsp;DPG计算公式如下所示：<br>

<center>$\begin{aligned}
\nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta}) &=\left.\sum_{s \in \mathcal{S}} \rho^{\mu_{\theta}}(s) \nabla_{\boldsymbol{\theta}} \mu(s, \boldsymbol{\theta}) f(s, a)\right|_{a=\mu(s, \boldsymbol{\theta})} \\
&=\mathbb{E}_{S_{t} \sim \rho^{\mu}}\left[\left.\nabla_{\boldsymbol{\theta}} \mu\left(S_{t}, \boldsymbol{\theta}\right) \nabla_{a} q\left(S_{t}, a\right)\right|_{a=\mu\left(S_{t}, \boldsymbol{\theta}\right)}\right]
\end{aligned}\tag{12.28}$</center><br>

其中，$\left.q\left(a, S_{t}\right)\right|_{a=\mu\left(S_{t}, \mathbf{\theta}\right)}$是基于确定性策略的动作值函数；有的书中会将$\nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta}) \text { 记为 } \nabla_{\boldsymbol{\theta}} J\left(\mu^{\theta}\right)$或$\nabla_{\theta} J\left(\boldsymbol{\theta}^{\mu}\right), \text { 以 } \boldsymbol{\theta}^{\mu} \text { 明确表示这是一个关于确定性策略 } \mu \text { 的参数 } \mathbf{\theta}$。<br>

&emsp;&emsp;DPG是SPG定理在策略方差趋向于0时的极限情况。比较SPG和DPG的策略梯度计算公式，可以发现它们的差异在于SPG中多一个log项，同时期望也不同，这些差异本质上是因为DPG不对动作求期望。<br>
&emsp;&emsp;**（2）异策略SPG与DPG**<br>
&emsp;&emsp;异策略SPG计算公式如下所示：<br>

<center>$\nabla J_{\beta}(\boldsymbol{\theta})=\mathbb{E}_{S_{t} \sim \rho^{\beta}, A_{t} \sim \beta}\left[\rho_{t} \nabla \log \pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right) q_{\pi_{\theta}}\left(S_{t}, A_{t}\right)\right]\tag{12.29}$</center><br>

其中，$ \pi  为目标策略，  \beta  为行为策略，有的书中会将  \beta  也直接表示为参数化策略函数  \beta_{\theta} ; J_{\beta} 
 (\boldsymbol{\theta}) $ $表示遵循行为策略  \beta  进行采样的，关于策略参数  \boldsymbol{\theta}  的目标函数;  \rho_{t}  为重要性采样权重:$ <br>
 
 <center>$\rho_{t}=\frac{\pi\left(A_{t} \mid S_{t}, \boldsymbol{\theta}\right)}{\beta\left(A_{t} \mid S_{t}\right)}$</center><br>
 
 异策略DPG计算公式如下所示：<br>
 
 <center>$\nabla J_{\beta}(\boldsymbol{\theta})=\mathbb{E}_{S_{t} \sim \rho^{\beta}}\left[\left.\nabla_{\boldsymbol{\theta}} \mu\left(S_{t}, \boldsymbol{\theta}\right) \nabla_{a} q\left(S_{t}, a\right)\right|_{a=\mu\left(S_{t}, \boldsymbol{\theta}\right)}\right]\tag{12.30}$</center><br>
 
 $\text { 有的书中会用 } \nabla_{\theta} J_{\beta}\left(\boldsymbol{\theta}^{\mu}\right) \text { 或 } \nabla_{\mathbf{\theta}} J_{\beta}\left(\mu^{\theta}\right) \text { 来明确表示这一个关于确定性策略参数 } \boldsymbol{\theta}^{\mu} \text { 的异策略 } \mathrm{DPG}$实际上，由于DPG都需要采用异策略，所以异策略DPG就直接被称为DPG。<br>
 &emsp;&emsp;对比于异策略SPG式(12,29)，式(12,30)在求解DPG时不需要重要性采样权重，这是因为重要性采样是用简单的概率分布估计复杂的概率分布，而确定性策略的动作为确定值，不存在这样一个概率分布；此外，确定性策略的值函数评估通常采用的是Q-learning算法，会忽略重要性采样权重。<br>
&emsp;&emsp;$基于近似动作值函数  \hat{q}(s, a, \mathbf{w}),  构建 DPG 方程如下所示 :$<br>

<center>$\nabla_{\theta} \hat{J}_{\beta}(\boldsymbol{\theta})=\mathbb{E}_{S_{t} \sim \rho^{\beta}}\left[\left.\nabla_{\theta} \mu\left(S_{t}, \boldsymbol{\theta}\right) \nabla_{a} \hat{q}\left(S_{t}, a, \mathbf{w}\right)\right|_{a=\mu\left(S_{t}, \mathbf{\theta}\right)}\right]\tag{12.31}$</center><br>

基于DPG的AC框架参数更新方程如下所示：<br>


 <center>$ \delta_{t}=R_{t+1}+\gamma \hat{q}\left(S_{t+1}, \mu\left(S_{t+1}, \boldsymbol{\theta}\right), \mathbf{w}\right)-\hat{q}\left(S_{t}, A_{t}, \mathbf{w}\right) $<br>
$ \mathbf{w}_{t+1}=\mathbf{w}_{t}+\alpha^{\mathrm{w}} \delta_{t} \nabla_{\mathrm{w}} \hat{q}\left(S_{t}, A_{t}, \mathbf{w}\right) 
评论家$<br>
 $\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_{t}+\left.\alpha^{\theta} \nabla_{\theta} \mu\left(S_{t}, \boldsymbol{\theta}\right) \nabla_{a} \hat{q}\left(S_{t}, a, \mathbf{w}\right)\right|_{a=\mu\left(S_{t}, \boldsymbol{\theta}\right)} 
行动者\tag{12.32}$</center><br>
 
此外，策略函数参数的更新也可以记为TD误差的形式：<br>

<center>$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_{t}+\alpha^{\theta} \nabla_{\theta} \mu\left(S_{t}, \boldsymbol{\theta}\right) \nabla_{a} \delta_{t}$</center><br>

 &emsp;&emsp;实际上，DPG训练起来并不稳定，模型的参数初始化对训练效果有着较大影响，需要多次尝试，有时奖励收敛一段时间后又会迅速下降，出现周期性变化。一种有效的解决方法就是引入深度学习方法，这一算法将在后面章节中说明。<br>
 
### 12.8 实验结果与分析

#### 12.8.1 实验环境设置
&emsp;&emsp;为了体现在某些函数近似的问题中，最好的近似策略可能是一个随机策略。而基于动作价值函数的方法没有一种自然的途径求解随机最优策略，REINFORCE作为一个随机梯度方法有很强的收敛性。为了说明随机策略的优点，引入短走廊网格世界环境，如图12.4所示。<br>
<center><img src='./第十二章图片/图12.4.png'></center>
<center>图12.4&ensp; 短走廊网格世界环境图</center><br>
&emsp;短走廊网格世界环境与大部分网格环境一样，每步的收益是-1，对于三个非终止状态都有两个动作可供选择：向左或者向右。特殊的是：第一个状态向左走会保持原地不动，而在第二个状态执行的动作会导致向相反的方向移动。<br>
&emsp;&emsp;本章对短走廊环境以及gym的CartPole-v0环境进行了实验&emsp;<br>

#### 12.8.2 REINFORCE算法代码

In [1]:
import argparse
import gym
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import matplotlib.pyplot as plt

parser = argparse.ArgumentParser(description='PyTorch REINFORCE example')
parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
                    help='discount factor (default: 0.99)')
parser.add_argument('--seed', type=int, default=543, metavar='N',
                    help='random seed (default: 543)')
parser.add_argument('--render', action='store_true',
                    help='render the environment')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='interval between training status logs (default: 10)')
args = parser.parse_args()


env = gym.make('CartPole-v0')
env.seed(args.seed)
torch.manual_seed(args.seed)


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)


policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()


def finish_episode():
    R = 0
    policy_loss = []
    rewards = []
    for r in policy.rewards[::-1]:
        R = r + args.gamma * R
        rewards.insert(0, R)
    rewards = torch.tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + eps)
    for log_prob, reward in zip(policy.saved_log_probs, rewards):
        policy_loss.append(-log_prob * reward)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]

def plot(epi, run_time):

    plt.title('Training')
    plt.xlabel('Episode')
    plt.ylabel('Run Time')
    plt.plot(epi, run_time, color='red')
    plt.show()

def main():
    running_reward = 10
    running_rewards = []
    i_episodes = []
    for i_episode in range(10000):
        state, ep_reward = env.reset(), 0
        for t in range(1, 10000):  # Don't infinite loop while learning
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            if args.render:
                env.render()
            policy.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        finish_episode()
        running_rewards.append(running_reward)
        i_episodes.append(i_episode)
        if i_episode % args.log_interval == 0:

            print('Episode {}\tLast length: {:2f}\tAverage length: {:.2f}'.format(
                i_episode, ep_reward, running_reward))
        if (i_episode % 1000 == 0):
            plot(i_episodes, running_rewards)
    np.save(f"putu", running_rewards)
if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'gym'

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tqdm import tqdm

def true_value(p):
    """ True value of the first state
    Args:
        p (float): probability of the action 'right'.
    Returns:
        True value of the first state.
        The expression is obtained by manually solving the easy linear system
        of Bellman equations using known dynamics.
    """
    return (2 * p - 4) / (p * (1 - p))

class ShortCorridor:
    """
    Short corridor environment, see Example 13.1
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.state = 0

    def step(self, go_right):
        """
        Args:
            go_right (bool): chosen action
        Returns:
            tuple of (reward, episode terminated?)
        """
        if self.state == 0 or self.state == 2:
            if go_right:
                self.state += 1
            else:
                self.state = max(0, self.state - 1)
        else:
            if go_right:
                self.state -= 1
            else:
                self.state += 1

        if self.state == 3:
            # terminal state
            return 0, True
        else:
            return -1, False

def softmax(x):
    t = np.exp(x - np.max(x))
    return t / np.sum(t)

class ReinforceAgent:
    """
    ReinforceAgent that follows algorithm
    'REINFORNCE Monte-Carlo Policy-Gradient Control (episodic)'
    """
    def __init__(self, alpha, gamma):
        # set values such that initial conditions correspond to left-epsilon greedy
        self.theta = np.array([-1.47, 1.47])
        self.alpha = alpha
        self.gamma = gamma
        # first column - left, second - right
        self.x = np.array([[0, 1],
                           [1, 0]])
        self.rewards = []
        self.actions = []

    def get_pi(self):
        h = np.dot(self.theta, self.x)
        t = np.exp(h - np.max(h))
        pmf = t / np.sum(t)
        # never become deterministic,
        # guarantees episode finish
        imin = np.argmin(pmf)
        epsilon = 0.05

        if pmf[imin] < epsilon:
            pmf[:] = 1 - epsilon
            pmf[imin] = epsilon

        return pmf

    def get_p_right(self):
        return self.get_pi()[1]

    def choose_action(self, reward):
        if reward is not None:
            self.rewards.append(reward)

        pmf = self.get_pi()
        go_right = np.random.uniform() <= pmf[1]
        self.actions.append(go_right)

        return go_right

    def episode_end(self, last_reward):
        self.rewards.append(last_reward)

        # learn theta
        G = np.zeros(len(self.rewards))
        G[-1] = self.rewards[-1]

        for i in range(2, len(G) + 1):
            G[-i] = self.gamma * G[-i + 1] + self.rewards[-i]

        gamma_pow = 1

        for i in range(len(G)):
            j = 1 if self.actions[i] else 0
            pmf = self.get_pi()
            grad_ln_pi = self.x[:, j] - np.dot(self.x, pmf)
            update = self.alpha * gamma_pow * G[i] * grad_ln_pi

            self.theta += update
            gamma_pow *= self.gamma

        self.rewards = []
        self.actions = []

class ReinforceBaselineAgent(ReinforceAgent):
    def __init__(self, alpha, gamma, alpha_w):
        super(ReinforceBaselineAgent, self).__init__(alpha, gamma)
        self.alpha_w = alpha_w
        self.w = 0

    def episode_end(self, last_reward):
        self.rewards.append(last_reward)

        # learn theta
        G = np.zeros(len(self.rewards))
        G[-1] = self.rewards[-1]

        for i in range(2, len(G) + 1):
            G[-i] = self.gamma * G[-i + 1] + self.rewards[-i]

        gamma_pow = 1

        for i in range(len(G)):
            self.w += self.alpha_w * gamma_pow * (G[i] - self.w)

            j = 1 if self.actions[i] else 0
            pmf = self.get_pi()
            grad_ln_pi = self.x[:, j] - np.dot(self.x, pmf)
            update = self.alpha * gamma_pow * (G[i] - self.w) * grad_ln_pi

            self.theta += update
            gamma_pow *= self.gamma

        self.rewards = []
        self.actions = []

def trial(num_episodes, agent_generator):
    env = ShortCorridor()
    agent = agent_generator()

    rewards = np.zeros(num_episodes)
    for episode_idx in range(num_episodes):
        rewards_sum = 0
        reward = None
        env.reset()

        while True:
            go_right = agent.choose_action(reward)
            reward, episode_end = env.step(go_right)
            rewards_sum += reward

            if episode_end:
                agent.episode_end(reward)
                break

        rewards[episode_idx] = rewards_sum

    return rewards

def example_13_1():
    epsilon = 0.05
    fig, ax = plt.subplots(1, 1)

    # Plot a graph
    p = np.linspace(0.01, 0.99, 100)
    y = true_value(p)
    ax.plot(p, y, color='red')

    # Find a maximum point, can also be done analytically by taking a derivative
    imax = np.argmax(y)
    pmax = p[imax]
    ymax = y[imax]
    ax.plot(pmax, ymax, color='green', marker="*", label="optimal point: f({0:.2f}) = {1:.2f}".format(pmax, ymax))

    # Plot points of two epsilon-greedy policies
    ax.plot(epsilon, true_value(epsilon), color='magenta', marker="o", label="epsilon-greedy left")
    ax.plot(1 - epsilon, true_value(1 - epsilon), color='blue', marker="o", label="epsilon-greedy right")

    ax.set_ylabel("Value of the first state")
    ax.set_xlabel("Probability of the action 'right'")
    ax.set_title("Short corridor with switched actions")
    ax.set_ylim(ymin=-105.0, ymax=5)
    ax.legend()

    plt.savefig('../images/example_13_1.png')
    plt.close()

def figure_13_1():
    num_trials = 100
    num_episodes = 1000
    gamma = 1
    agent_generators = [lambda : ReinforceAgent(alpha=2e-4, gamma=gamma),
                        lambda : ReinforceAgent(alpha=2e-5, gamma=gamma),
                        lambda : ReinforceAgent(alpha=2e-3, gamma=gamma)]
    labels = ['alpha = 2e-4',
              'alpha = 2e-5',
              'alpha = 2e-3']

    rewards = np.zeros((len(agent_generators), num_trials, num_episodes))

    for agent_index, agent_generator in enumerate(agent_generators):
        for i in tqdm(range(num_trials)):
            reward = trial(num_episodes, agent_generator)
            rewards[agent_index, i, :] = reward

    plt.plot(np.arange(num_episodes) + 1, -11.6 * np.ones(num_episodes), ls='dashed', color='red', label='-11.6')
    for i, label in enumerate(labels):
        plt.plot(np.arange(num_episodes) + 1, rewards[i].mean(axis=0), label=label)
    plt.ylabel('total reward on episode')
    plt.xlabel('episode')
    plt.legend(loc='lower right')

    plt.savefig('../images/figure_13_1.png')
    plt.close()

def figure_13_2():
    num_trials = 100
    num_episodes = 1000
    alpha = 2e-4
    gamma = 1
    agent_generators = [lambda : ReinforceAgent(alpha=alpha, gamma=gamma),
                        lambda : ReinforceBaselineAgent(alpha=alpha*10, gamma=gamma, alpha_w=alpha*100)]
    labels = ['Reinforce without baseline',
              'Reinforce with baseline']

    rewards = np.zeros((len(agent_generators), num_trials, num_episodes))

    for agent_index, agent_generator in enumerate(agent_generators):
        for i in tqdm(range(num_trials)):
            reward = trial(num_episodes, agent_generator)
            rewards[agent_index, i, :] = reward

    plt.plot(np.arange(num_episodes) + 1, -11.6 * np.ones(num_episodes), ls='dashed', color='red', label='-11.6')
    for i, label in enumerate(labels):
        plt.plot(np.arange(num_episodes) + 1, rewards[i].mean(axis=0), label=label)
    plt.ylabel('total reward on episode')
    plt.xlabel('episode')
    plt.legend(loc='lower right')

    plt.savefig('../images/figure_13_2.png')
    plt.close()

if __name__ == '__main__':
    example_13_1()
    figure_13_1()
    figure_13_2()

#### 12.8.3 REINFORCE with bathline算法代码

In [ ]:

import torch.nn as nn
import torch.nn.functional as F
import gym
import torch
from torch.distributions import Categorical
import torch.optim as optim
from copy import deepcopy
import numpy as np
import argparse
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter
from torch.nn.utils import clip_grad_norm_
render = False
class Policy(nn.Module):
    def __init__(self,n_states, n_hidden, n_output):
        super(Policy, self).__init__()
        self.linear1 = nn.Linear(n_states, n_hidden)
        self.linear2 = nn.Linear(n_hidden, n_output)

 #这是policy的参数
        self.reward = []
        self.log_act_probs = []
        self.Gt = []
        self.sigma = []
#这是state_action_func的参数
        # self.Reward = []
        # self.s_value = []

    def forward(self, x):
        x = F.relu(self.linear1(x))
        output = F.softmax(self.linear2(x), dim= 1)
        # self.act_probs.append(action_probs)
        return output
env = gym.make('CartPole-v0')
n_states = env.observation_space.shape[0]
n_actions = env.action_space.n
policy = Policy(n_states, 128, n_actions)
s_value_func = Policy(n_states, 128, 1)
alpha_theta = 1e-3
optimizer_theta = optim.Adam(policy.parameters(), lr=alpha_theta)
gamma = 0.99
seed = 1
env.seed(seed)
torch.manual_seed(seed)
live_time = []
def plot(epi, run_time):

    plt.title('Training')
    plt.xlabel('Episode')
    plt.ylabel('Run Time')
    plt.plot(epi, run_time)
    plt.show()
if __name__ == '__main__':
    running_reward = 10
    i_episodes = []
    for i_episode in range(1, 10000):
        state, ep_reward = env.reset(), 0
        if render: env.render()
        policy_loss = []
        s_value = []
        state_sequence = []
        log_act_prob = []
        for t in range(10000):
            state = torch.from_numpy(state).unsqueeze(0).float()  # 在第0维增加一个维度，将数据组织成[N , .....] 形式
            state_sequence.append(deepcopy(state))
            action_probs = policy(state)
            m = Categorical(action_probs)
            action = m.sample()
            m_log_prob = m.log_prob(action)
            log_act_prob.append(m_log_prob)
            # policy.log_act_probs.append(m_log_prob)
            action = action.item()
            next_state, re, done, _ = env.step(action)
            if render: env.render()
            policy.reward.append(re)
            ep_reward += re
            if done:
                break
            state = next_state
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        i_episodes.append(i_episode)
        if i_episode % 10 == 0:
            print('Episode {}\tLast length: {:2f}\tAverage length: {:.2f}'.format(
                i_episode, ep_reward, running_reward))
        live_time.append(running_reward)
        R = 0
        Gt = []

        # get Gt value
        for r in policy.reward[::-1]:
            R = r + gamma * R
            Gt.insert(0, R)
        # update step by step
        for i in range(len(Gt)):
            G = Gt[i]
            V = s_value_func(state_sequence[i])
            delta = G - V

            # update value network
            alpha_w = 1e-3  # 初始化

            optimizer_w = optim.Adam(s_value_func.parameters(), lr=alpha_w)
            optimizer_w.zero_grad()
            policy_loss_w = -delta
            policy_loss_w.backward(retain_graph=True)
            clip_grad_norm_(policy_loss_w, 0.1)
            optimizer_w.step()

            # update policy network
            optimizer_theta.zero_grad()
            policy_loss_theta = - log_act_prob[i] * delta
            policy_loss_theta.backward(retain_graph=True)
            clip_grad_norm_(policy_loss_theta, 0.1)
            optimizer_theta.step()
        del policy.log_act_probs[:]
        del policy.reward[:]
        if (i_episode % 1000 == 0):
            plot(i_episodes, live_time)
    np.save(f"withB", live_time)
    #policy.plot(live_time)

### 3.7.4实验结果分析

<center><img src='./第十二章图片/图12.5.png'></center>
<center>图12.5&ensp; 短走廊环境结果图	</center><br>

<center><img src='./第十二章图片/图12.6.png'></center>
<center>图12.6&ensp; CartPole环境结果图</center><br>
&emsp;&emsp;从图12.5和图12.6可以看出，在两种环境下带基线的REINFORCE算法效果都优于REINFORCE算法。在短走廊环境下带基线的REINFORCE算法最终也收敛在-11.6处，与REINFORCE算法一致，但收敛的更快，在100个情节后即可收敛。在CartPole环境中，带基线的REINFORCE算法大约在600个情节稳定在第200个时间步。对这两个环境下的表现进行比较，在短走廊环境下两个算法收敛之后都较为稳定。在CartPole环境中，两个算法在整个训练过程都波动较大，带基线的REINFORCE算法更为明显。这是因为与短走廊环境相比较，CartPole环境更复杂。尽管REINFORCE算法在处理简单环境，如短走廊环境，有较好效果，但在处理复杂问题时则不尽如意。为了解决此问题，本书在后续章节将引入基于行动者-评论家框架的DDPG、TD3算法等。<br>